In [1]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd


In [2]:
#browser = webdriver.Edge()
browser = webdriver.Chrome("webdriver/chromedriver")

/var/folders/5p/_jlphrrx7hzf7hq0z69f1_nc0000gn/T/ipykernel_11325/3983540099.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("webdriver/chromedriver")


In [6]:
browser.quit()

In [10]:
#browser.switch_to.window(browser.window_handles[-1])

In [3]:
def go_to_search_page(browser):
    link = "https://www.telecompaper.com/search"
    browser.get(link)
    sleep(5)

In [4]:
def collect_news_links(browser):
    hrefs=[]
    elements_news_link = browser.find_elements(By.XPATH, '//div[@class="column is-12"]/a')

    for element in elements_news_link:
        hrefs.append(element.get_attribute("href"))
    
    print("Total search items: " + str(len(hrefs)))    
    return hrefs

In [5]:
def collect_headlines(browser):
    headline=[]
    elements_news_text = browser.find_elements(By.XPATH, "//div[@class='column is-12']/a/div/div/h2")
    
    for element in elements_news_text:
        headline.append(element.text)
        
    return headline

In [6]:
def extract_news(hrefs,n):
    news = []
    news_date = []
    news_country = []
    news_headline = []
    
    for href in hrefs[:int(n)]:
        browser.get(href)
        sleep(3)

        news_paragraph = browser.find_element(By.XPATH, '//p[@class="is-abstract"]')
        news.append(news_paragraph.text)

        country = browser.find_element(By.XPATH, '//div[@class="is-meta mb-2"]/span[1]')
        news_country.append(country.text)

        date = browser.find_element(By.XPATH, '//div[@class="is-meta mb-2"]/span[2]')
        news_date.append(date.text)
        
        headline = browser.find_element(By.XPATH, '//h2[@class="is-title mb-2"]')
        news_headline.append(headline.text)
    
    
    data = {'Region': news_country, 'Date': news_date, 'Headline':news_headline , 'News': news, 'Link':hrefs[0:len(news)]}
    df = pd.DataFrame(data)
    return df

In [7]:
headers = ['Region' , 'Date', 'Headline', 'News', 'Link']
result_data = pd.DataFrame(columns=headers)

In [9]:
print("Search and add filters.")
y="A"

while 1:
    go_to_search_page(browser)
    y = input('Type "Y" to continue, "E" to exit: ')
    if y=="Y" or y=="y":
        #headlines = collect_headlines(browser)
        news_links = collect_news_links(browser)
        n = input("Out of " + str(len(news_links)) + ", how many news you want to extract: ")
        
        scrapped_data = extract_news(news_links,n)
        result_data = pd.concat([result_data, scrapped_data], ignore_index=True)
        
        
    elif y=="E" or y=="e":
        browser.quit()
        break


    


Search and add filters.
Total search items: 6
Total search items: 3
Total search items: 5
Total search items: 1
Total search items: 1
Total search items: 4
Total search items: 10


In [14]:
print("total no of news collected: " + str(len(result_data)))

total no of news collected: 30


In [11]:
result_data.to_excel("data.xlsx")